In [1]:
import os
import pickle

import pandas as pd
import numpy as np

os.chdir(os.path.join("..", ".."))

from solution.constants import (
    REVIEWS_PATH,
    PRODUCTS_PATH,
    RETURN_REASONS_PATH,
    RETURNS_PATH,
    TEST_PATH,
    BAD_REVIEWS_GROUPED_BY_PRODUCT_ID,
)
from solution.fasttext import embed_sentences

# Imports

# Embdeddings

## Reviews

In [2]:
reviews_df = pd.read_parquet(REVIEWS_PATH)
reviews_df.head()

,order_item_id,product_id,customer_id,review_text,shop_id,rating,date_created
0,19068657,641695ab1f06465947d1e4e26f61ed308145dce718d9b6...,0aaefdb4bd01799d678f7fcd453e23c597804eb006f1ce...,товар потерян,c2f61742e5f0ef0a74c5fcbeff7ff659def91bf4b5f1e1...,1,2023-10-17 17:26:42.874
1,11130626,2c0818e03896a3e27fd8ec01ba9d7ae80d97673575a847...,851513a761b89239303515254d0e9c22a8474dd2df60b8...,,757db91a80964d58a2b0d26bffd641bcbe142aae4f7f47...,5,2023-07-27 10:06:40.711
2,8992153,d00a0a981e5618c4f96802e79c205ef2175b141edff368...,96659e77ab5e9b52ecbf110eed661c3fe742fc42443d50...,,8f0f97e140e126a4404a09eb1e14a53b7c742701c4290a...,5,2023-07-03 14:47:59.316
3,11078898,d0d9bad11fa6fedcf5c699055137a36f4c4c81d722b5fd...,321667e5d0ada2fa7f2356076f88a3848e596cde7bd120...,"Rahmat, juda yoqdi!",61182f39851829ca78c919a83ecbfa045fc0686bff16d0...,5,2023-07-30 12:26:59.864
4,32080499,4edd99b2e2a203a18cf24b6a2b134833ef1f74cf1940c0...,d67c451b69c7f706cedf6d4a87f1180ea62acd57c94a29...,16гб работает хорошо,02cca3803b564ede11ccf9f303c9910b39c532061e7a8c...,5,2024-01-19 06:02:57.971


### Bad reviews

In [3]:
bad_reviews_df = reviews_df[reviews_df.rating < 4].reset_index(drop=True)
bad_reviews_df.head(3)

,order_item_id,product_id,customer_id,review_text,shop_id,rating,date_created
0,19068657,641695ab1f06465947d1e4e26f61ed308145dce718d9b6...,0aaefdb4bd01799d678f7fcd453e23c597804eb006f1ce...,товар потерян,c2f61742e5f0ef0a74c5fcbeff7ff659def91bf4b5f1e1...,1,2023-10-17 17:26:42.874
1,8150477,bdd57b3266b0681ceeae2c2110a475050d264f582bb8e7...,4e3c1775f2acdc861bf51c95256546df3d6731305389de...,шланг тешик экан . маслахат бермайман,77523aa0395b6ee089984c28fd543755244df3ba6adba2...,1,2023-06-23 09:30:32.777
2,1912117,78499f983a337e22849e3106844923f547858d69fe3e73...,bfb208bd78f18d8af0ea504e0be0da2e4f77f70f1c5c32...,yaxshi ekan.,784818021864dcfd07659616848650cd479d6efe957ad3...,3,2023-02-26 06:38:21.852


In [4]:
bad_reviews_df.shape

(183726, 7)

In [5]:
sum(bad_reviews_df.review_text.isnull())

0

In [6]:
bad_reviews_grouped_by_product_id = bad_reviews_df.groupby("product_id").indices

In [7]:
reviews_embs = embed_sentences(bad_reviews_df.review_text)

  0%|          | 0/183726 [00:00<?, ?it/s]

In [8]:
reviews_embs.shape

(183726, 300)

In [9]:
bad_reviews_embs_by_id = {}

for product_id, review_indeces in bad_reviews_grouped_by_product_id.items():
    embs = reviews_embs[review_indeces]
    product_emb = np.mean(embs, axis=0)

    bad_reviews_embs_by_id[product_id] = product_emb

In [10]:
with open(BAD_REVIEWS_GROUPED_BY_PRODUCT_ID, 'wb') as f:
    pickle.dump(bad_reviews_embs_by_id, f)